In [9]:
%load_ext autoreload
%autoreload 2

import dask
import dask.dataframe as dd
import glob
import pandas as pd
from pathlib import Path 

import numpy as np
import scipy.stats as sps
from sklearn import linear_model

from distributed import Client
client = Client()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/marek/NGIMS-hp/env/lib/python3.8/site-packages/distributed/node.py:240: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41305 instead
  warnings.warn(


In [10]:
PATH_MAVEN = Path("/home/marek")
PATH_NGI = Path(PATH_MAVEN) / "maven" / "data" / "sci" / "ngi"
PATH_NGI_L2 = Path(PATH_NGI) / "l2"

NA_VALUES = [" ", "-999", np.inf, "Inf", "inf"]

In [11]:
year = 2015

In [12]:
test_dir = Path(PATH_NGI_L2, f"{year}/02/*02[0-2][1-9]*.csv")

In [13]:
meta_cols = {
    "orbit": int,
    "alt": float,
    "species": str,
    "abundance": float,
    "t_unix": float
}

In [14]:
ddf = dd.read_csv(
    test_dir, 
    assume_missing=True, 
    usecols=["orbit", "alt", "species", "abundance", "t_unix"],
    include_path_column=True,
    dtype=meta_cols,
    na_values = [" ", "-999", np.inf, "Inf", "inf"]
)

In [15]:
ddf.head()

,t_unix,orbit,alt,species,abundance,path
0,1.423626e+09,713,495.0434,Ar,0.0,/home/marek/maven/data/sci/ngi/l2/2015/02/mvn_...
1,1.423626e+09,713,493.1250,Ar,0.0,/home/marek/maven/data/sci/ngi/l2/2015/02/mvn_...
2,1.423626e+09,713,491.2105,Ar,0.0,/home/marek/maven/data/sci/ngi/l2/2015/02/mvn_...
3,1.423626e+09,713,489.2999,Ar,0.0,/home/marek/maven/data/sci/ngi/l2/2015/02/mvn_...
4,1.423626e+09,713,487.3931,Ar,0.0,/home/marek/maven/data/sci/ngi/l2/2015/02/mvn_...


In [16]:
temp_ddf = dd.read_csv(
    test_dir, 
    assume_missing=True, 
    usecols=list(meta_cols.keys()),
    dtype=meta_cols,
    na_values = NA_VALUES
)

In [17]:
temp_ddf.head()

,t_unix,orbit,alt,species,abundance
0,1.423626e+09,713,495.0434,Ar,0.0
1,1.423626e+09,713,493.1250,Ar,0.0
2,1.423626e+09,713,491.2105,Ar,0.0
3,1.423626e+09,713,489.2999,Ar,0.0
4,1.423626e+09,713,487.3931,Ar,0.0


In [18]:
def make_orbit_path_map(ddf, orb_span):
    orb_path_map = ddf[["orbit", "path"]].drop_duplicates().compute()
    orb_orb_map = {
        x: list(
            range(x - orb_span//2, x + orb_span//2 + 1)
        ) for x in orb_path_map["orbit"]
    }
    orb_filename_map = {
        x: orb_path_map["path"][orb_path_map["orbit"].isin(orb_orb_map[x])].tolist() 
        for x in orb_orb_map.keys()
}
    return orb_filename_map

def IO_orb(orbdata,io='I') -> pd.DataFrame:
    minalt = orbdata['alt'].min()
    peri_t = orbdata[orbdata['alt']==minalt]['t_unix'].unique()
    #if len(peri_t)>1:
    #    sys.exit('Non-unique time found at periapse '+str(orbdata['orbit'].unique()))
    #else:
    if io == 'I':
        return orbdata[orbdata['t_unix']<=peri_t[0]]
    elif io =='O':
        return orbdata[orbdata['t_unix']>peri_t[0]]
    else:
        return orbdata

In [19]:
DEFAULT_ORBIT_SPAN = 10
#from src.homopause_rolling_orbits import make_orbit_path_map, IO_orb
orb_path_map = make_orbit_path_map(ddf, DEFAULT_ORBIT_SPAN)

In [20]:
import scipy.stats as sps
import scipy.integrate as spi

def exo_Ar_int(CO2,Ar,alt,exsp=['CO2'],ArXsec=[3.e-15],\
           Ntop=[0.,0.],taufrange=[0.,5]):
    '''
    Calculate exobase altitude from species profiles
    (BE SURE TO USE INBOUND)
    Integrates down from some initial altitude to periapse
    Determines where num_density*coll_x-sec = 1 for exo altitude

    Inputs
    ------
    exsp: list, species to use in calculation
    ArXsec: list, collisional cross section for each species in exsp
    Ntop: list, column density above top for each species

    Outputs
    -------
    exo: float, exobase altitude
    fitTau: fit parameters of Tau profile (see scipy.stats.linregress)


    **TO DO:
    cite cross section values, should extrapolate ones that don't reac tau=1
    '''
    #convert alts to cm
    #orb_df = orb_df[orb_df['abundance_CO2']>0]
    xsec = dict(zip(exsp,ArXsec))
    altkm = np.array(alt)*1.e+5
    Tau_sp_dz = np.zeros((len(exsp),len(altkm)))  #initialize Tau/z
    for i,s in enumerate(xsec): #loop through species to use
        #colname = 'abundance_'+s
        #if colname in orb_df.columns: #check given species in DF
        Tau_sp_dz[i] = CO2*xsec[s] #calc n*sigma
        #else:
        #    print 'has no column '+colname
        #    return np.NaN
    Tau_tot_dz = np.sum(Tau_sp_dz,axis=0) #add each sp n*sigma together
    Tau_int = spi.cumtrapz(Tau_tot_dz,altkm*-1) #n*dz*sigma=N*sigma=Tau
    altmids = ((altkm[1:] + altkm[:-1]) / 2)/1.e+5 #gid mid alts in km
    findTau1 = np.where((Tau_int>taufrange[0])&(Tau_int<taufrange[-1])) #cond to find Tau=1
    if len(Tau_int[findTau1])<5: #warn if fitting line to only a few pts
        if np.max(Tau_int) < 1:
            print('Never reaches tau=1, <'+str(int(altmids[-1]))+'?')
            return np.NaN, (np.NaN,np.NaN,np.NaN,np.NaN,np.NaN)
        else:
            print('Has <5 points near tau=',taufrange[0],'-',taufrange[-1])
            return np.NaN, (np.NaN,np.NaN,np.NaN,np.NaN,np.NaN)
    fitTau = sps.linregress(altmids[findTau1],Tau_int[findTau1])
    exo = (1-fitTau[1])/fitTau[0] #find alt where Tau=1
    return exo,fitTau #return exobase altitude
    

In [21]:
@dask.delayed()
def exo_files(files):
    files = list(files)
    temp_ddf = dd.read_csv(
        files, 
        assume_missing=True, 
        usecols=list(meta_cols.keys()),
        dtype=meta_cols,
        na_values = NA_VALUES
    )
    temp_ddf = temp_ddf.map_partitions(IO_orb, meta=temp_ddf)
    temp_ddf = temp_ddf[(temp_ddf["abundance"] > 0.) & (temp_ddf["species"].isin(["Ar", "CO2"]))]
    df = temp_ddf.compute()
    norbs = len(df["orbit"].unique())
    df = df.pivot_table(values=["abundance"], index=["orbit", "alt", "species"]).unstack()
    df = df.dropna()
    df = df.droplevel("orbit")
    df = df.sort_index(ascending=False)
    Ar = df[("abundance", "Ar")]
    CO2 = df[("abundance", "CO2")]
    alt = np.array(df.index.tolist())
    exo = exo_Ar_int(CO2, Ar, alt, taufrange=[0,2])
    return exo, norbs

In [23]:
for orb, files in orb_path_map.items():
    exo, n = exo_files(list(files)).compute()
    print(exo[0], n)

173.19189412227917 6
173.65741256795874 7
172.29558343341944 8
171.53465937594896 9
172.0272658060411 10
172.0601456439167 11
174.07249594038854 11
174.4775039624486 11
173.74230235691297 11
172.11633895310348 11
174.08245906192252 11
173.27734255760842 11
173.40123028567618 11
175.38015404519822 11
175.88402727277904 11
175.65744265354223 11
177.850961787167 11
176.39894205289906 11
176.78753162104792 11
175.50740422844214 11
177.34982646600443 11
179.02490860209323 11
179.8899209776895 11
181.15450277648006 11
177.37631366668558 11
180.2249370927412 11
180.89419953834798 11
179.98847204237106 11
180.46167142576778 11
179.74122579854773 11
178.0378753874643 10
178.98915313474404 9
178.79979446676913 8
177.18350005162014 7
177.86985595410832 6
174.23639961036955 6
175.39993970090904 7
174.75237944547027 8
175.0289590618742 9
173.7606193044824 10
174.09555039322606 11
173.25969639052067 11
174.21026825429533 11
174.27985217094337 11
173.51099091882284 11
173.11214357385307 11
172.053414

In [15]:
df.compute()

NameError: name 'df' is not defined

In [16]:
df.get_partition(1).head()

NameError: name 'df' is not defined

In [17]:
df.head().compute()

NameError: name 'df' is not defined